# iBioFoundry #3 - Cell Plater - OT2
author: Camillo Moschner | version: 2.0 | date: 23.01.2022 | license = 

# 1- Metadata

In [1]:
metadata = {
    'protocolName': 'iBF_3_Cplater',
    'author': 'Camillo Moschner <cm967@cam.ac.uk> / <camillo.moschner@gmail.com>',
    'description': 'Automated Liquid Handling for Cell Plating using OT2',
    'apiLevel': '2.12',
    'Date': '10.05.2022',
    'pipette_configuration':{'left':'p20_single_gen2',
                             'right':'p300_single_gen2'}
    }

## Cell Plating Master Inputs
The following cell requires changing between experimental setups (and, for simplicity for the user, is supposed to be the only part of the protocol that requires any modification).

In [2]:
script_mode = 'simulation' # 'simulation' or 'execution'
experiment_name = 'sTUFPlibrary'

# competent cells
comp_cell_tadded_format = 'opentrons_96_aluminumblock_generic_pcr_strip_200ul' # opentrons_96_aluminumblock_biorad_wellplate_200ul
comp_cell_tadded_column_usage = 'all' # allows you to selectively only use 'odd'- or 'even'-numbered columns or 'all'; useful for 0.2 ml tube usage
comp_cell_fill_start_position_well = 'A1'
# plates for cell plating
cell_plating_plate_format = 'corning_6_wellplate_16.8ml_flat' # 'corning_12_wellplate_6.9ml_flat' #'corning_24_wellplate_3.4ml_flat'
# plating volumes
plating_volumes = [140.0] # units: µl

# 2- Import Statements

In [3]:
import numpy as np
from itertools import chain
import pandas as pd
from IPython.display import display, clear_output, Audio, display
from copy import deepcopy
import time
from datetime import date
import math
from copy import deepcopy
import os
import re

from iBioFoundry_helper import *
from opentrons import protocol_api, execute
from opentrons import simulate

In [4]:
# establish sole Jupyter Notebook control 
if script_mode == 'execution':
    try:
        os.system("systemctl stop opentrons-robot-server")
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    except:
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in exectution mode!\nYou should see the lights turn on and hear the gantry homing!")
elif script_mode == 'simulation':
    protocol = simulate.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in simulation mode!")
protocol.set_rail_lights(True)
protocol.home()

/Users/camillomoschner/.opentrons/robot_settings.json not found. Loading defaults
/Users/camillomoschner/.opentrons/deck_calibration.json not found. Loading defaults


You are operating this script in simulation mode!


# 3- Function Definitions

In [5]:
def smart_transfer_liquid(transfer_vol, aspiration_pos,dispensation_pos,
                          pick_up_tip=True, touch_tip_before=True,
                          cyclical_mix_before=None,mix_before_reps=2,mix_rate=0.5,
                          asp_bot_clearance=1.5,dispens_bot_clearance=1.5,
                          asp_rate=0.8,disp_rate=0.5,
                          cyclical_mix_after=None,mix_after=None,
                          dispens_top_offset=-2, blow_out=False, air_gap=False, touch_tip=True, touch_miniscus=None,
                          drop_tip=True):
    """
    Liquid transfer that automatically identifies which tip to use.
    """
    if (transfer_vol >= p20_left.min_volume) & (transfer_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (transfer_vol > p300_right.min_volume) & (transfer_vol <= p300_right.max_volume):
        pipette_to_use = p300_right
    else:
        print(f"WARNING: transfer volume not manageable")
    if pick_up_tip==True:
        pipette_to_use.pick_up_tip()
    if cyclical_mix_before != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_before[0]):
            pipette_to_use.aspirate(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[2]))
            pipette_to_use.dispense(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[3]))
    if mix_before_reps != 0:
        pipette_to_use.mix(mix_before_reps,transfer_vol*0.8,aspiration_pos.bottom(asp_bot_clearance),mix_rate)
    # aspiration
    pipette_to_use.aspirate(transfer_vol,aspiration_pos.bottom(asp_bot_clearance), asp_rate)
    #protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
    pipette_to_use.move_to(aspiration_pos.top())
    #protocol.max_speeds['Z'] = None # reset z axis speed
    if air_gap != False:
        pipette_to_use.air_gap(air_gap)
    protocol.delay(seconds=1)
    if touch_tip_before == True:
        pipette_to_use.touch_tip(v_offset=-2, speed=150)
    # dispensation
    pipette_to_use.dispense(transfer_vol,dispensation_pos.bottom(dispens_bot_clearance),disp_rate)
    if cyclical_mix_after != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_after[0]):
            pipette_to_use.aspirate(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[2]))
            pipette_to_use.dispense(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[3]))
    if mix_after != None:
        pipette_to_use.mix(mix_after[0],mix_after[1], dispensation_pos.bottom(mix_after[2]),mix_rate)
    pipette_to_use.move_to(dispensation_pos.top())
    protocol.delay(seconds=2)
    if blow_out == True:
        pipette_to_use.blow_out(dispensation_pos.top(dispens_top_offset))
    if touch_tip == True:
        pipette_to_use.touch_tip(v_offset=dispens_top_offset, speed=150)
    if touch_miniscus != None:
        protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
        pipette_to_use.move_to(dispensation_pos.bottom(touch_miniscus))
        protocol.max_speeds['Z'] = None # reset z axis speed
    if drop_tip==True:
        pipette_to_use.drop_tip()

def smart_distribute_liquid(single_distr_vol, aspiration_pos,dispensation_pos_list,
                            asp_bot_clearance=1.1,dispens_bot_clearance=1.8,
                            asp_flow_rate = 1,disp_flow_rate = 1, new_disposal_volume=None,
                            touch_tip_bool=True):
    """
    Liquid distribution from one aspiration well into multiple destination wells that automatically identifies which pipette to use.
    """
    if (single_distr_vol[0] >= p20_left.min_volume) & (single_distr_vol[0] <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (single_distr_vol[0] > p300_right.min_volume):
        pipette_to_use = p300_right
    # save original/default flow rates
    original_asp_flow_rate = pipette_to_use.flow_rate.aspirate
    original_disp_flow_rate = pipette_to_use.flow_rate.dispense
    # change flow rates if desired
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate *asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate *disp_flow_rate
    # change well_bottom_clearances if desired
    pipette_to_use.well_bottom_clearance.aspirate = asp_bot_clearance
    pipette_to_use.well_bottom_clearance.dispense = dispens_bot_clearance
    # execute
    if new_disposal_volume != None:
        pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list, 
                                  touch_tip=touch_tip_bool,disposal_volume=new_disposal_volume)
    else:
        pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list, 
                              touch_tip=touch_tip_bool)
    # set well_bottom_clearances back to default
    pipette_to_use.well_bottom_clearance.dispense = 1
    pipette_to_use.well_bottom_clearance.aspirate = 1
    # set flow rates back to default
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate

---
# 4- Deck assignments

In [6]:
"""Deck creation"""
deck_slot_list =list(np.arange(1,12))
deck_slot_list.append('bin')
deck_slot_df = pd.DataFrame( np.flip(np.array(deck_slot_list).reshape(4,3), axis=0) )

## i. Labware
Resource: [Opentrons Labware Library Webpage](https://labware.opentrons.com/?_gl=1*1iyc1t4*_ga*MTU1MTM1NzU5MS4xNjE2OTczMzY5*_ga_GNSMNLW4RY*MTYzMDYxNzY1NS4yMC4wLjE2MzA2MTc2NTUuMA..&_ga=2.110416899.767557364.1630617656-1551357591.1616973369)
### a. Modules

In [7]:
temp_mod_name  = 'temperature module gen2'
temp_mod = protocol.load_module(temp_mod_name,'3')
deck_slot_df.iloc[3,2] = temp_mod_name

### b. Racks & Plates

In [8]:
tiprack20a = protocol.load_labware('opentrons_96_tiprack_20ul','11')
tiprack20b = protocol.load_labware('opentrons_96_tiprack_20ul','10')
tiprack300a = protocol.load_labware('opentrons_96_tiprack_300ul','7')
# tuberacks & plates
comp_cell_plate = temp_mod.load_labware(comp_cell_tadded_format,'3') 
CP_plate1 = protocol.load_labware(cell_plating_plate_format,'8')
CP_plate2 = protocol.load_labware(cell_plating_plate_format,'9')
CP_plate3 = protocol.load_labware(cell_plating_plate_format,'6')

CP_plates = (CP_plate1, CP_plate2, CP_plate3)

In [9]:
SOC_rack = protocol.load_labware('opentrons_6_tuberack_falcon_50ml_conical','4') 
labware_list = [tiprack20a,tiprack20b, tiprack300a,
                comp_cell_plate, CP_plate1, CP_plate2, CP_plate3, SOC_rack]
_ =[update_labware(deck_slot_df, labware_item) for labware_item in labware_list]
# visually inspect deck
print("\nFinal OT-2 Deck Configuration:");(deck_slot_df)


Final OT-2 Deck Configuration:


,0,1,2
0,10-opentrons_96_tiprack_20ul,11-opentrons_96_tiprack_20ul,bin
1,7-opentrons_96_tiprack_300ul,8-corning_6_wellplate_16.8ml_flat,9-corning_6_wellplate_16.8ml_flat
2,4-opentrons_6_tuberack_falcon_50ml_conical,5,6-corning_6_wellplate_16.8ml_flat
3,1,2,temperature module gen2


## ii. Pipettes

Resource: [Opentrons Pipettes API Webpage](https://docs.opentrons.com/v2/new_pipette.html)

In [10]:
p20_left = protocol.load_instrument('p20_single_gen2','left', tip_racks=[tiprack20a,tiprack20b])
p300_right = protocol.load_instrument('p300_single_gen2','right', tip_racks=[tiprack300a])

## iii. Assembly info integration

In [11]:
assemblies_info_dir = f'1_GGA{os.path.sep}assemblies_summary.csv'
assemblies_info_df = pd.read_csv(assemblies_info_dir)
assembly_no = len(assemblies_info_df['assembly_ID'])

In [12]:
assemblies_info_df

,assembly_ID,assembly_description,Part 1: Promoter,Part 2: RBS,Part 3: CDS,Part 4: Terminator,Part 5: LVL1 Destination Vector,inter_stock_ID,assembly_tube,assembly_date,transformation_tube,transformation_date,SOC_vol,CP_plate_format,CP_plate_no,CP_pos_per_round,CP_wells,colonies
0,pCMCW37,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,A1,09/05/2022,A1,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,['A3'],NaN
1,pCMCW38,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,B1,09/05/2022,A2,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,['A2'],NaN
2,pCMCW39,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,C1,09/05/2022,A3,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,['A1'],NaN
3,pCMCW40,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,D1,09/05/2022,A4,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,['B3'],NaN
4,pCMCW41,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0034,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0034|CIDAR_LVL0_CDS_mVen...,E1,09/05/2022,A5,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,['B2'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,pCMCW104,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0032...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,B12,09/05/2022,F8,10/05/2022,125,corning_6_wellplate_16.8ml_flat,11,1,['A2'],NaN
68,pCMCW105,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0033...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,C12,09/05/2022,F9,10/05/2022,125,corning_6_wellplate_16.8ml_flat,11,1,['A1'],NaN
69,pCMCW106,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0033...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,D12,09/05/2022,F10,10/05/2022,125,corning_6_wellplate_16.8ml_flat,11,1,['B3'],NaN
70,pCMCW107,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0034...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0034,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0034|CIDAR_LVL0_CDS_mVen...,E12,09/05/2022,F11,10/05/2022,125,corning_6_wellplate_16.8ml_flat,11,1,['B2'],NaN


---
# 5- LH Step Calculations 
Create summary statements of assembly requirements, deck capacity, and calculated pipetting operations:

In [13]:
no_of_CP_plates, required_wells = len(CP_plates), len(plating_volumes)*assembly_no
well_avail_per_plate = len(Plate(identify_plate(cell_plating_plate_format, 'plate_format')).well_list)
assemblies_per_plate = int(well_avail_per_plate/len(plating_volumes))
well_avail_per_round = no_of_CP_plates*well_avail_per_plate
assemblies_avail_per_round = int(well_avail_per_round/len(plating_volumes))
plating_per_round_checkup_list = list(range(0,assembly_no, assemblies_avail_per_round))
column_no_per_plate = Plate(identify_plate(cell_plating_plate_format, 'plate_format')).columns
# swath = no of complete plating rows
swaths = int(Plate(identify_plate(cell_plating_plate_format, 'plate_format')).rows/len(plating_volumes))
no_required_plates,no_required_rounds = math.ceil(required_wells/well_avail_per_plate), math.ceil(required_wells/well_avail_per_round)
print(f"""Your assembly:
->   total no constructs: {assembly_no}
->           {len(plating_volumes)} dilutions: {[vol for vol in plating_volumes]} µl
->      required well no: {required_wells} wells""")
print(f"""\nPlates:
->                              format: {cell_plating_plate_format}
->                         wells/plate: {well_avail_per_plate}
->                      swath(s)/plate: {swaths}
->                no of plates on deck: {no_of_CP_plates}
-> total available wells/plating round: {well_avail_per_round}""")
print(f"""\nResult:
->               required plates: {no_required_plates}
->     required plating round(s): {no_required_rounds}\n""")

Your assembly:
->   total no constructs: 72
->           1 dilutions: [140.0] µl
->      required well no: 72 wells

Plates:
->                              format: corning_6_wellplate_16.8ml_flat
->                         wells/plate: 6
->                      swath(s)/plate: 2
->                no of plates on deck: 3
-> total available wells/plating round: 18

Result:
->               required plates: 12
->     required plating round(s): 4



In [14]:
comp_cell_plate_obj = Plate(identify_plate(comp_cell_tadded_format, 'plate_format'),only_columns=comp_cell_tadded_column_usage)
comp_cell_plate_obj.layout = extend_fill_plate_df_with_list(comp_cell_plate_obj.layout,assemblies_info_df['assembly_ID'].to_list(),
                                                            fill_start_position=comp_cell_fill_start_position_well, fill_first='rows')
print(f"__Layout of {comp_cell_plate.parent}__")
comp_cell_plate_obj.layout

__Layout of Temperature Module GEN2 on 3__


,1,2,3,4,5,6,7,8,9,10,11,12
A,pCMCW37,pCMCW38,pCMCW39,pCMCW40,pCMCW41,pCMCW42,pCMCW43,pCMCW44,pCMCW45,pCMCW46,pCMCW47,pCMCW48
B,pCMCW49,pCMCW50,pCMCW51,pCMCW52,pCMCW53,pCMCW54,pCMCW55,pCMCW56,pCMCW57,pCMCW58,pCMCW59,pCMCW60
C,pCMCW61,pCMCW62,pCMCW63,pCMCW64,pCMCW65,pCMCW66,pCMCW67,pCMCW68,pCMCW69,pCMCW70,pCMCW71,pCMCW72
D,pCMCW73,pCMCW74,pCMCW75,pCMCW76,pCMCW77,pCMCW78,pCMCW79,pCMCW80,pCMCW81,pCMCW82,pCMCW83,pCMCW84
E,pCMCW85,pCMCW86,pCMCW87,pCMCW88,pCMCW89,pCMCW90,pCMCW91,pCMCW92,pCMCW93,pCMCW94,pCMCW95,pCMCW96
F,pCMCW97,pCMCW98,pCMCW99,pCMCW100,pCMCW101,pCMCW102,pCMCW103,pCMCW104,pCMCW105,pCMCW106,pCMCW107,pCMCW108
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0


## CP plates and plating well allocation

In [15]:
# create multi-well plates objects for each cell plating step
multiwell_plates_objs = [Plate(identify_plate(cell_plating_plate_format, 'plate_format'),name=f"CP_Plate_{x}") for x in range(no_required_plates)]
name_decipher_list = list(divide_list_into_chunks(multiwell_plates_objs,no_of_CP_plates))
# create list of assemblies*dilutions
column_chunked_single_plating_list = list(divide_list_into_chunks(assemblies_info_df['assembly_ID'].to_list(), column_no_per_plate*swaths))
assembly_lists_per_plate = [list(chain.from_iterable([[x+f"_{int(plating_volumes[index])}"
                                                       for index in range(len(plating_volumes))]
                                                      for x in column_chunked_single_plating_list[plate_idx] ]))
                            for plate_idx in range(len(column_chunked_single_plating_list))]
# check that created multi-well objects correspond to calculated plating lists
if not len(multiwell_plates_objs) == len(assembly_lists_per_plate):
    print(f"""WARNING: number of multiwell_plates_objs and assembly_lists_per_plate is not the same.
    Only use plates whose len(rows) is a multiple of the number of dilutions you want to perform.""")
print(f"__Cell Plating (CP) Plates in Deck Positions {', '.join([plate_pos.parent for plate_pos in CP_plates])} through {no_required_rounds} Plating Rounds__")
# fill each multi-well plate object with the assembly*dilution that is supposed to pipetted there
for idx, plate_obj in enumerate(multiwell_plates_objs):
    reversefill_CPplate_df_fromTopRight_with_list(plate_obj.layout, assembly_lists_per_plate[idx],len(plating_volumes))
    print(plate_obj.name)
    display(plate_obj.layout)

__Cell Plating (CP) Plates in Deck Positions 8, 9, 6 through 4 Plating Rounds__
CP_Plate_0


,1,2,3
A,pCMCW39_140,pCMCW38_140,pCMCW37_140
B,pCMCW42_140,pCMCW41_140,pCMCW40_140


CP_Plate_1


,1,2,3
A,pCMCW45_140,pCMCW44_140,pCMCW43_140
B,pCMCW48_140,pCMCW47_140,pCMCW46_140


CP_Plate_2


...

,1,2,3
A,pCMCW93_140,pCMCW92_140,pCMCW91_140
B,pCMCW96_140,pCMCW95_140,pCMCW94_140


CP_Plate_10


,1,2,3
A,pCMCW99_140,pCMCW98_140,pCMCW97_140
B,pCMCW102_140,pCMCW101_140,pCMCW100_140


CP_Plate_11


,1,2,3
A,pCMCW105_140,pCMCW104_140,pCMCW103_140
B,pCMCW108_140,pCMCW107_140,pCMCW106_140


-> save summary of all cell plating events across all CP plates in easily-readable format by executing the following two cells (one to create the output and one to save it):

In [16]:
%%capture cap --no-stderr
print(f"__Cell Plating (CP) Plates in Deck Positions {', '.join([plate_pos.parent for plate_pos in CP_plates])} through {no_required_rounds} Plating Rounds__")
for idx, plate_obj in enumerate(multiwell_plates_objs):
    print(f"{plate_obj.name}\n{plate_obj.layout}\n")

In [17]:
with open('cell_plating_pos_summary.txt','w+') as myfile:
    myfile.read()
    myfile.seek(0)
    myfile.write(cap.stdout)
    myfile.truncate()

In [18]:
# identify what assembly goes into what plate and what well(s)
if 'CP_plate_no' not in assemblies_info_df.columns:
    cell_plating_positions = [[find_df_coordinates_containing(plate_obj.layout,assembly_id,statement=False) 
                           for plate_obj in multiwell_plates_objs] 
                          for assembly_id in assemblies_info_df['assembly_ID']]
    cell_plating_info_df = pd.DataFrame([[x for x in enumerate(pos_info) if isinstance(x[1],list)][0]
                                         for pos_info in cell_plating_positions], columns=['CP_plate_no','CP_wells'])
    assemblies_info_df['CP_plate_format'] = [cell_plating_plate_format for id in assemblies_info_df['assembly_ID']]

    assemblies_info_df = pd.concat([assemblies_info_df,cell_plating_info_df],axis=1)
    assemblies_info_df.insert(len(assemblies_info_df.columns)-1, column='CP_pos_per_round', value=plate_index_mapping(assemblies_info_df.CP_plate_no.to_list(),no_of_CP_plates))
else: # in case you run the script twice / have already saved the spreadsheet w/ the new plating info before 
    assemblies_info_df.CP_wells = assemblies_info_df.CP_wells.apply(lambda x: re.findall(r'\w\d+', x))
assemblies_info_df['colonies'] = [np.nan for id in assemblies_info_df['assembly_ID']]

In [19]:
assemblies_info_df.head(5)

,assembly_ID,assembly_description,Part 1: Promoter,Part 2: RBS,Part 3: CDS,Part 4: Terminator,Part 5: LVL1 Destination Vector,inter_stock_ID,assembly_tube,assembly_date,transformation_tube,transformation_date,SOC_vol,CP_plate_format,CP_plate_no,CP_pos_per_round,CP_wells,colonies
0,pCMCW37,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,A1,09/05/2022,A1,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,[A3],NaN
1,pCMCW38,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,B1,09/05/2022,A2,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,[A2],NaN
2,pCMCW39,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,C1,09/05/2022,A3,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,[A1],NaN
3,pCMCW40,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,D1,09/05/2022,A4,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,[B3],NaN
4,pCMCW41,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0034,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0034|CIDAR_LVL0_CDS_mVen...,E1,09/05/2022,A5,10/05/2022,125,corning_6_wellplate_16.8ml_flat,0,0,[B2],NaN


## Update summary Spreadsheets

update_assembly_summary_answer = input("Do you want to update the GGA summary spreadsheet with the cell plating information? (yes / no)\n answer:")
if update_assembly_summary_answer == 'yes':
    assemblies_info_df.to_csv(assemblies_info_dir,index=False)
    print(f"\n -> Your GGA summary spreadsheet for experiment \'{experiment_name}\' has been updated with cell plating information!\n")
else:
    print(f"\n -> Ensure proper recording of your results! Tracking your plated cells can be strenuous.\n")

---
# 6- Execution
## i. Cell plating

In [21]:
print("Cell plating...")
counter = 0

for idx, ID in enumerate(assemblies_info_df['assembly_ID'].to_list()):
    current_ID_info = assemblies_info_df.loc[assemblies_info_df.assembly_ID == ID]
    source_well_name, target_well_names = current_ID_info.transformation_tube.iloc[0], current_ID_info.CP_wells.iloc[0]
    target_CP_plate_no = current_ID_info.CP_plate_no.iloc[0]
    per_round_target_CP_plate_no = current_ID_info.CP_pos_per_round.iloc[0]
    # exchange cell plating multi-well plates whenever they one round of plating fills up all available wells
    if idx in plating_per_round_checkup_list:
        plate_load_question = 'no'
        while plate_load_question != 'yes':
            plate_load_question = input(f"""\nHave you loaded the {len([plate for plate in name_decipher_list[counter]])}x {cell_plating_plate_format},
             - {f" &{os.linesep}             - ".join([f"CP_plate_{plate.name}" for plate in name_decipher_list[counter]])} 
                               in position(s) {', '.join([plate_pos.parent for plate_pos in CP_plates])} ? (yes/no) \n  answer:""")
            print("\n")
            if plate_load_question == 'skip':
                break
        counter += 1
    if plate_load_question != 'skip':
        # update user on liquid handling status
        print(f"""  {idx}/{assembly_no} - {ID}: transformation_plate \'{source_well_name}\' -> cell_plating_plate \'{target_CP_plate_no}\': wells {target_well_names}
                                                                               {list(plating_volumes)} µl """)
        smart_distribute_liquid(plating_volumes, 
                                comp_cell_plate[source_well_name], # aspiration bottom clearance,
                                [CP_plates[per_round_target_CP_plate_no].wells_by_name()[well_name] # plating/dispensation bottom clearance
                                 for well_name in target_well_names],
                                asp_bot_clearance=2.,dispens_bot_clearance=12,
                                asp_flow_rate = 0.8,disp_flow_rate = 2,
                                touch_tip_bool=False,new_disposal_volume=5)

Cell plating...
0



Have you loaded the 3x corning_6_wellplate_16.8ml_flat,
             - CP_plate_CP_Plate_0 &
             - CP_plate_CP_Plate_1 &
             - CP_plate_CP_Plate_2 
                               in position(s) 8, 9, 6 ? (yes/no) 
  answer: yes




  0/72 - pCMCW37: transformation_plate 'A1' -> cell_plating_plate '0': wells ['A3']
                                                                               [140.0] µl 
  1/72 - pCMCW38: transformation_plate 'A2' -> cell_plating_plate '0': wells ['A2']
                                                                               [140.0] µl 
  2/72 - pCMCW39: transformation_plate 'A3' -> cell_plating_plate '0': wells ['A1']
                                                                               [140.0] µl 
  3/72 - pCMCW40: transformation_plate 'A4' -> cell_plating_plate '0': wells ['B3']
                                                                               [140.0] µl 
  4/72 - pCMCW41: transformation_plate 'A5' -> cell_plating_plate '0': wells ['B2']
                                                                               [140.0] µl 
  5/72 - pCMCW42: transformation_plate 'A6' -> cell_plating_plate '0': wells ['B1']
                                       


Have you loaded the 3x corning_6_wellplate_16.8ml_flat,
             - CP_plate_CP_Plate_3 &
             - CP_plate_CP_Plate_4 &
             - CP_plate_CP_Plate_5 
                               in position(s) 8, 9, 6 ? (yes/no) 
  answer: n



Have you loaded the 3x corning_6_wellplate_16.8ml_flat,
             - CP_plate_CP_Plate_3 &
             - CP_plate_CP_Plate_4 &
             - CP_plate_CP_Plate_5 
                               in position(s) 8, 9, 6 ? (yes/no) 
  answer: n


...


Have you loaded the 3x corning_6_wellplate_16.8ml_flat,
             - CP_plate_CP_Plate_6 &
             - CP_plate_CP_Plate_7 &
             - CP_plate_CP_Plate_8 
                               in position(s) 8, 9, 6 ? (yes/no) 
  answer: yes




  36/72 - pCMCW73: transformation_plate 'D1' -> cell_plating_plate '6': wells ['A3']
                                                                               [140.0] µl 
  37/72 - pCMCW74: transformation_plate 'D2' -> cell_plating_plate '6': wells ['A2']
                                                                               [140.0] µl 
  38/72 - pCMCW75: transformation_plate 'D3' -> cell_plating_plate '6': wells ['A1']
                                                                               [140.0] µl 
  39/72 - pCMCW76: transformation_plate 'D4' -> cell_plating_plate '6': wells ['B3']
                                                                               [140.0] µl 
  40/72 - pCMCW77: transformation_plate 'D5' -> cell_plating_plate '6': wells ['B2']
                                                                               [140.0] µl 
  41/72 - pCMCW78: transformation_plate 'D6' -> cell_plating_plate '6': wells ['B1']
                                 


Have you loaded the 3x corning_6_wellplate_16.8ml_flat,
             - CP_plate_CP_Plate_9 &
             - CP_plate_CP_Plate_10 &
             - CP_plate_CP_Plate_11 
                               in position(s) 8, 9, 6 ? (yes/no) 
  answer: no



Have you loaded the 3x corning_6_wellplate_16.8ml_flat,
             - CP_plate_CP_Plate_9 &
             - CP_plate_CP_Plate_10 &
             - CP_plate_CP_Plate_11 
                               in position(s) 8, 9, 6 ? (yes/no) 
  answer: yes




  54/72 - pCMCW91: transformation_plate 'E7' -> cell_plating_plate '9': wells ['A3']
                                                                               [140.0] µl 
  55/72 - pCMCW92: transformation_plate 'E8' -> cell_plating_plate '9': wells ['A2']
                                                                               [140.0] µl 
  56/72 - pCMCW93: transformation_plate 'E9' -> cell_plating_plate '9': wells ['A1']
                                                                               [140.0] µl 
  57/72 - pCMCW94: transformation_plate 'E10' -> cell_plating_plate '9': wells ['B3']
                                                                               [140.0] µl 
  58/72 - pCMCW95: transformation_plate 'E11' -> cell_plating_plate '9': wells ['B2']
                                                                               [140.0] µl 
  59/72 - pCMCW96: transformation_plate 'E12' -> cell_plating_plate '9': wells ['B1']
                              

# Shutdown

In [ ]:
# allDone()
try:
    p20_left.drop_tip()
except:
    pass
try:
    p300_right.drop_tip()
except:
    pass
temp_mod.deactivate()
protocol.home()

In [ ]:
# complete tip usage check:
_=[display(show_rack_usage(labware)[1]) for labware in labware_list if 'Opentrons 96 Tip Rack' in str(labware)]